In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2022-ai-w6p1/sample_submission.csv
/kaggle/input/2022-ai-w6p1/pca_test.csv
/kaggle/input/2022-ai-w6p1/pca_train.csv
/kaggle/input/2022-ai-w6p1/train_label.csv


In [2]:
import random
import torch
import numpy as np
import pandas as pd


seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

# GPU 사용하기

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print('Device:', device)
print('Current cuda device:', torch.cuda.current_device())
print('Count of using GPUs:', torch.cuda.device_count())

Device: cuda
Current cuda device: 0
Count of using GPUs: 1


# 데이터 불러오기

In [4]:
X_train = pd.read_csv('../input/2022-ai-w6p1/pca_train.csv')
X_train

,0,1,2,3,4,5,6,7,8,9,...,246,247,248,249,250,251,252,253,254,255
0,-6.227718,-2.070960,0.674094,-0.742135,0.309149,-4.006772,-0.664624,0.533987,0.024594,-0.195856,...,-2.114192,-0.605770,0.491399,-0.166693,1.289197,-0.403863,0.238731,2.166737,-0.106487,1.486114
1,-6.081077,-1.960213,0.846482,-0.932816,-0.050208,-4.082826,-0.449032,0.647842,0.417809,0.073842,...,-1.000466,-1.175968,0.299093,-1.187048,-0.499101,0.738016,0.632842,0.850430,-2.211427,1.083040
2,-6.394956,-2.954897,1.600033,-0.479463,-1.217065,-3.986472,-0.812934,0.472509,1.536917,1.989765,...,-1.703212,0.713673,0.330699,0.036953,-0.204944,0.930597,-0.693151,-0.572443,0.835598,-0.751483
3,-6.681355,-2.055031,1.144864,-0.517872,-0.168057,-3.929032,-0.718259,0.555827,0.096472,0.724261,...,0.434870,1.571784,-1.266089,1.584173,-0.290301,0.068907,-1.148258,1.904273,0.322937,-0.318100
4,-5.716713,-2.163729,0.877930,-0.605268,0.082372,-4.261466,-0.679734,0.739369,0.490033,0.546437,...,0.626112,-0.435030,2.112485,1.101894,1.746205,-0.445612,-1.518138,0.299177,-0.020655,0.234824
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27666,-6.047007,-2.072704,0.668214,-0.254932,0.099894,-1.477478,-0.703836,-0.327026,0.504125,0.222820,...,-1.449840,-0.050902,-1.240334,-0.750522,-1.168852,0.048051,0.432534,-0.003478,-0.609472,-0.348196
27667,-6.569392,-1.932956,0.602381,0.253941,-0.341952,4.064471,-1.502157,-2.467482,0.281797,0.812378,...,0.721433,1.197452,-0.700384,-3.483315,-0.371596,0.478422,-1.537939,2.030344,-1.080110,-0.409972
27668,-6.232087,-1.834862,2.097659,-0.566552,-0.288520,-4.034954,0.457435,0.315611,0.981373,0.483152,...,0.662459,-1.293272,0.067110,0.357502,-0.155424,-0.658622,2.016063,-0.359339,0.993638,0.130081
27669,-6.387326,-0.387639,-1.090173,1.499431,0.629389,31.163396,-3.186039,-12.114586,-0.543894,-1.469931,...,-0.838120,0.584897,-1.355164,-1.358166,1.344522,-0.526878,-0.230260,-0.459776,2.507944,-1.379322


In [5]:
y_train = pd.read_csv('../input/2022-ai-w6p1/train_label.csv')
y_train

,id,label
0,0,136
1,1,90
2,2,112
3,3,165
4,4,73
...,...,...
27666,27666,9
27667,27667,132
27668,27668,18
27669,27669,41


## X_train 정규화 + tensor로 바꾸기

In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_train

array([[-0.16764337, -0.11283343,  0.04929722, ...,  2.14773098,
        -0.1056013 ,  1.47440317],
       [-0.16369595, -0.10679955,  0.06190416, ...,  0.8429707 ,
        -2.19303037,  1.0745059 ],
       [-0.17214522, -0.16099359,  0.11701218, ..., -0.56742165,
         0.82864692, -0.74556154],
       ...,
       [-0.16776096, -0.09996995,  0.15340412, ..., -0.35618734,
         0.9853716 ,  0.12905576],
       [-0.17193982, -0.02111998, -0.07972553, ..., -0.45574277,
         2.48708031, -1.36845308],
       [-0.17005028, -0.11768155,  0.07936767, ..., -0.80047655,
         0.24567059,  2.30036452]])

In [7]:
X_train = torch.FloatTensor(X_train).to(device)
X_train

tensor([[-0.1676, -0.1128,  0.0493,  ...,  2.1477, -0.1056,  1.4744],
        [-0.1637, -0.1068,  0.0619,  ...,  0.8430, -2.1930,  1.0745],
        [-0.1721, -0.1610,  0.1170,  ..., -0.5674,  0.8286, -0.7456],
        ...,
        [-0.1678, -0.1000,  0.1534,  ..., -0.3562,  0.9854,  0.1291],
        [-0.1719, -0.0211, -0.0797,  ..., -0.4557,  2.4871, -1.3685],
        [-0.1701, -0.1177,  0.0794,  ..., -0.8005,  0.2457,  2.3004]],
       device='cuda:0')

## y_train 형태 만들기 + tensor로 바꾸기

In [8]:
y_train = y_train['label']
y_train

0        136
1         90
2        112
3        165
4         73
        ... 
27666      9
27667    132
27668     18
27669     41
27670    114
Name: label, Length: 27671, dtype: int64

In [9]:
y_train = torch.LongTensor(y_train).to(device)
y_train

tensor([136,  90, 112,  ...,  18,  41, 114], device='cuda:0')

## shape 확인하기

In [10]:
print(X_train.shape)
print(y_train.shape)

torch.Size([27671, 256])
torch.Size([27671])


In [11]:
np.shape(y_train.unique())

torch.Size([198])

In [12]:
print(y_train.unique())

tensor([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
         28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
         42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
         56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
         70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
         84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
         98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
        112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
        126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139,
        140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153,
        154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167,
        168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 1

# 모델 정의하기

In [13]:
linear1 = torch.nn.Linear(256,128,bias=True)
linear5 = torch.nn.Linear(128,198,bias=True)
sigmoid = torch.nn.ReLU()

In [14]:
model = torch.nn.Sequential(linear1,sigmoid,linear5,sigmoid).to(device)

In [15]:
model

Sequential(
  (0): Linear(in_features=256, out_features=128, bias=True)
  (1): ReLU()
  (2): Linear(in_features=128, out_features=198, bias=True)
  (3): ReLU()
)

# 모델 학습하기

In [16]:
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=1e-2)
#(regularization)weight_decay=0.5

In [17]:
for stop in range(30000):
    
    # 그래디언트 초기화
    optimizer.zero_grad()
    
    # Forward 계산
    hypothesis = model(X_train)
    # Error 계산
    cost = loss(hypothesis, y_train)
    
    # Backward 계산 
    cost.backward()
    # 가중치 갱신
    optimizer.step()

    if stop % 1000 == 0:
        print(stop, cost.item())


0 5.311746120452881
1000 4.299528121948242
2000 3.176389455795288
3000 2.7333896160125732
4000 2.526129961013794
5000 2.393305778503418
6000 2.2825820446014404
7000 2.161618947982788
8000 2.023939847946167
9000 1.9620476961135864
10000 1.9152858257293701
11000 1.8757083415985107
12000 1.8410277366638184
13000 1.8093863725662231
14000 1.7805674076080322
15000 1.7536388635635376
16000 1.7277811765670776
17000 1.7028539180755615
18000 1.6788029670715332
19000 1.654955267906189
20000 1.6309417486190796
21000 1.607796311378479
22000 1.5854148864746094
23000 1.5630724430084229
24000 1.5411767959594727
25000 1.5198571681976318
26000 1.4993996620178223
27000 1.479957103729248
28000 1.4614001512527466
29000 1.4434951543807983


In [18]:
(y_train == hypothesis.argmax(1)).sum()

tensor(19263, device='cuda:0')

# 테스트 데이터 불러오기

In [19]:
X_test = pd.read_csv('../input/2022-ai-w6p1/pca_test.csv')
X_test

,0,1,2,3,4,5,6,7,8,9,...,246,247,248,249,250,251,252,253,254,255
0,-6.220117,-2.277101,1.034768,-0.384605,-0.702699,-4.246655,-0.807866,0.572210,1.165823,1.152361,...,0.163893,-0.709582,0.498515,1.707836,-0.948594,-0.131648,-1.640947,0.515782,0.792422,0.788818
1,-6.450735,-2.273477,1.006427,-0.819267,-0.241652,-4.031211,-0.656995,0.636281,0.498062,0.542788,...,0.357484,-0.432345,-0.524281,-0.099300,-1.288575,-0.511968,0.500356,-1.395989,-0.721703,0.261530
2,-6.044194,-2.472646,1.285010,-0.368850,-0.363810,-3.983979,-0.674843,0.518772,0.815135,0.699784,...,1.176633,1.637722,0.263460,-1.154657,0.192604,-0.260725,-0.363973,0.123888,-0.461763,-0.140312
3,-7.379873,-1.032564,0.234467,-0.388409,0.355459,-0.352889,-0.938281,-0.946746,-0.396938,-0.847589,...,-0.445818,0.249247,0.423985,0.438247,0.646347,1.164281,0.142753,0.359090,0.837276,0.957077
4,-6.166922,-1.771786,0.663905,-0.671532,0.331461,-4.181923,-0.701542,0.661430,-0.033613,-0.043006,...,1.838244,0.647151,1.941593,2.228317,-2.104379,0.737671,-0.538868,-0.791611,-0.985132,0.292099
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13624,-6.513840,-1.394930,0.333204,-0.750949,0.594688,-2.635995,-0.934044,0.123369,-0.418801,-0.769193,...,-0.232304,-0.425656,-0.497238,0.473078,0.140824,-0.986948,-0.119994,1.339758,-0.874920,2.389680
13625,-6.106158,-1.552375,0.321694,-1.177582,0.602137,-4.203300,-0.633238,0.583460,-0.281046,-0.439632,...,0.275283,-1.469434,0.598442,0.322251,1.010329,0.473637,-0.509026,0.689395,0.380682,0.477235
13626,-5.916207,-2.285041,0.972209,-0.869802,-0.065306,-4.426993,-0.555087,0.755705,0.188009,0.763321,...,0.689430,-1.357263,-0.646554,-0.128856,0.063910,0.294675,0.687431,-0.235042,-0.097503,-1.266684
13627,-6.016793,-2.482594,1.318416,-0.495801,-0.424331,-4.067531,-0.634893,0.520420,0.972367,0.910569,...,0.528882,-1.928248,0.046155,-0.195394,-0.019051,-1.422435,-0.931999,-0.224663,0.726427,-0.626489


In [20]:
# 정규화
X_test = scaler.transform(X_test)

In [21]:
# 텐서로 변환
X_test = torch.FloatTensor(X_test).to(device)
X_test

tensor([[-0.1674, -0.1241,  0.0757,  ...,  0.5113,  0.7858,  0.7826],
        [-0.1736, -0.1239,  0.0736,  ..., -1.3837, -0.7157,  0.2595],
        [-0.1627, -0.1347,  0.0940,  ...,  0.1228, -0.4579, -0.1392],
        ...,
        [-0.1593, -0.1245,  0.0711,  ..., -0.2330, -0.0967, -1.2567],
        [-0.1620, -0.1353,  0.0964,  ..., -0.2227,  0.7204, -0.6216],
        [-0.1602, -0.1073,  0.0569,  ..., -0.3570, -1.2700,  1.7332]],
       device='cuda:0')

# 모델 평가하기

In [22]:
hypothesis = model(X_test)
hypothesis

tensor([[0.0000, 0.0073, 0.0000,  ..., 0.6889, 1.6290, 0.0000],
        [1.6177, 1.7286, 1.3308,  ..., 1.7660, 0.6158, 0.9242],
        [0.0000, 1.2712, 1.3794,  ..., 1.2966, 0.2679, 0.1240],
        ...,
        [0.7070, 2.4850, 1.9934,  ..., 0.9403, 0.3801, 0.4794],
        [0.0000, 0.9536, 1.3544,  ..., 0.5175, 0.0000, 0.3972],
        [0.0000, 0.4281, 1.2161,  ..., 0.5784, 0.0000, 0.0000]],
       device='cuda:0', grad_fn=<ReluBackward0>)

In [23]:
with torch.no_grad():
    
    hypothesis = model(X_test)
    predicted = hypothesis.argmax(1)
    print('\n Hypothesis: ', hypothesis, '\n Correct: ', predicted)


 Hypothesis:  tensor([[0.0000, 0.0073, 0.0000,  ..., 0.6889, 1.6290, 0.0000],
        [1.6177, 1.7286, 1.3308,  ..., 1.7660, 0.6158, 0.9242],
        [0.0000, 1.2712, 1.3794,  ..., 1.2966, 0.2679, 0.1240],
        ...,
        [0.7070, 2.4850, 1.9934,  ..., 0.9403, 0.3801, 0.4794],
        [0.0000, 0.9536, 1.3544,  ..., 0.5175, 0.0000, 0.3972],
        [0.0000, 0.4281, 1.2161,  ..., 0.5784, 0.0000, 0.0000]],
       device='cuda:0') 
 Correct:  tensor([113, 118,  68,  ...,  23,  68, 117], device='cuda:0')


In [24]:
predicted.unique()

tensor([  0,   1,   2,   3,   5,   6,   7,   8,  12,  13,  14,  15,  16,  17,
         18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,  30,  31,
         35,  37,  38,  40,  43,  44,  47,  49,  50,  52,  53,  54,  61,  62,
         63,  64,  65,  66,  67,  68,  69,  70,  75,  76,  77,  79,  80,  81,
         82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  98, 110, 111, 112,
        113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126,
        127, 128, 129, 130, 131, 132, 134, 135, 136, 144, 147, 155, 156, 157,
        158, 159, 160, 162, 164, 165, 166, 167, 168, 169, 172, 173, 174, 175,
        176, 177, 178, 179, 180, 181, 183, 192], device='cuda:0')

In [25]:
predicted = np.array(predicted.cpu())
predicted

array([113, 118,  68, ...,  23,  68, 117])

# 제출 파일 만들기

In [26]:
submit = pd.read_csv('../input/2022-ai-w6p1/sample_submission.csv')
submit

,id,label
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
13624,13624,0
13625,13625,0
13626,13626,0
13627,13627,0


In [27]:
submit['label'] = predicted
submit

,id,label
0,0,113
1,1,118
2,2,68
3,3,28
4,4,121
...,...,...
13624,13624,77
13625,13625,69
13626,13626,23
13627,13627,68


In [28]:
submit.to_csv('submit.csv',index=False)
submit

,id,label
0,0,113
1,1,118
2,2,68
3,3,28
4,4,121
...,...,...
13624,13624,77
13625,13625,69
13626,13626,23
13627,13627,68
